In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern, hog
from scipy.stats import moment
import mahotas
from skimage import measure
from sklearn.preprocessing import StandardScaler
import ast

In [ ]:
# Function to normalize an image
def normalize_image(image):
    return cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)


In [ ]:
# Function to resize an image
def resize_image(image, size):
    return cv2.resize(image, size)


In [ ]:
# Function to extract color moments
def extract_color_moments(image):
    moments_r = moment(image[:, :, 0].ravel(), moment=[1, 2, 3])
    moments_g = moment(image[:, :, 1].ravel(), moment=[1, 2, 3])
    moments_b = moment(image[:, :, 2].ravel(), moment=[1, 2, 3])
    return moments_r, moments_g, moments_b


In [ ]:
# Function to extract Haralick texture features
def extract_haralick_features(gray_image):
    glcm = graycomatrix(gray_image, [1], [0], 256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    entropy = graycoprops(glcm, 'ASM')[0, 0]
    return contrast, correlation, entropy


In [ ]:
# Function to extract LBP features
def extract_lbp_features(gray_image):
    lbp = local_binary_pattern(gray_image, 8, 1, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 9))
    return hist


In [ ]:
# Function to extract Zernike moments
def extract_zernike_moments(gray_image):
    return mahotas.features.zernike_moments(gray_image, radius=10, degree=8)


In [ ]:
# Function to extract Hu moments
def extract_hu_moments(gray_image):
    return cv2.HuMoments(cv2.moments(gray_image)).flatten()


In [ ]:
# Function to apply Gabor filters and extract features
def apply_gabor_filters(gray_image):
    gabor_features = []
    for theta in range(4):
        theta = theta / 4. * np.pi
        kernel = cv2.getGaborKernel((21, 21), 5.0, theta, 10.0, 0.5, 0, ktype=cv2.CV_32F)
        fimg = cv2.filter2D(gray_image, cv2.CV_8UC3, kernel)
        gabor_features.append(fimg.mean())
        gabor_features.append(fimg.var())
    return gabor_features


In [ ]:
# Function to extract contour-based features
def extract_contour_features(binary_image, gray_image):
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea) if contours else None
    if largest_contour is None:
        return None
    area = cv2.contourArea(largest_contour)
    perimeter = cv2.arcLength(largest_contour, True)
    x, y, w, h = cv2.boundingRect(largest_contour)
    aspect_ratio = float(w) / h
    contour_image = np.zeros_like(gray_image)
    cv2.drawContours(contour_image, [largest_contour], -1, (255), thickness=cv2.FILLED)
    eccentricity = measure.regionprops(measure.label(contour_image))[0].eccentricity
    hull = cv2.convexHull(largest_contour)
    hull_area = cv2.contourArea(hull)
    hull_perimeter = cv2.arcLength(hull, True)
    solidity = area / hull_area
    return area, perimeter, aspect_ratio, eccentricity, hull_area, hull_perimeter, solidity


In [ ]:
# Function to process images and extract features
def process_images(dataset_dir, output_dir, input_size):
    image_path = os.path.join(root, filename)
    image = cv2.imread(image_path)
    
    # Normalize and resize the image
    normalized_image = normalize_image(image)
    resized_image = resize_image(normalized_image, input_size)
    
    # Save augmented images
    save_augmented_images(resized_image, filename, subfolder_output_dir)

    # Convert the resized image to grayscale
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    
    # Extract features
    moments_r, moments_g, moments_b = extract_color_moments(image)
    contrast, correlation, entropy = extract_haralick_features(gray_image)
    lbp_features = extract_lbp_features(gray_image)
    zernike_moments = extract_zernike_moments(gray_image)
    hu_moments = extract_hu_moments(gray_image)
    gabor_features = apply_gabor_filters(gray_image)

    # Extract HSV histograms
    hsv_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2HSV)
    hist_h = cv2.calcHist([hsv_image], [0], None, [256], [0, 256]).flatten()
    hist_s = cv2.calcHist([hsv_image], [1], None, [256], [0, 256]).flatten()
    hist_v = cv2.calcHist([hsv_image], [2], None, [256], [0, 256]).flatten()

    # Extract GLCM properties
    glcm = graycomatrix(gray_image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)
    glcm_props = [graycoprops(glcm, prop).ravel()[0] for prop in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']]

    # Extract contour-based features
    binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    contour_features = extract_contour_features(binary_image, gray_image)
    
    if contour_features is not None:
        area, perimeter, aspect_ratio, eccentricity, hull_area, hull_perimeter, solidity = contour_features
        
      features = np.concatenate((
            ['Contrast': contrast, 'Dissimilarity': glcm_props[1], 'Homogeneity': glcm_props[2], 'Energy': glcm_props[3],
            'Correlation': glcm_props[4],'Area': area, 'Perimeter': perimeter,'Aspect_Ratio': aspect_ratio, 
            'Eccentricity': eccentricity, 'Hull_Area': hull_area, 'Hull_Perimeter': hull_perimeter, 'Solidity': solidity,]
            'Hue_Histogram': hist_h.tolist(), 'Saturation_Histogram': hist_s.tolist(), 'Value_Histogram': hist_v.tolist(),
            'Moments_R': moments_r.tolist(), 'Moments_G': moments_g.tolist(), 'Moments_B': moments_b.tolist(),
            'LBP_Features': lbp_features.tolist(),'Zernike_Moments': zernike_moments.tolist(), 'Hu_Moments': hu_moments.tolist(),
            'Gabor_Features': gabor_features
      ))
       

    

In [ ]:
# Load the scaler
scaler_path = r"C:\Users\User\Desktop\exprement\logistic_regression_scaler.pkl"
scaler = joblib.load(scaler_path)

# Standardize the features
features_scaled = scaler.transform(features.reshape(1, -1))

# Load the model
model_path =  r'C:\Users\User\Desktop\exprement\logistic_regression_model.pkl'
model = joblib.load(model_path)

# Make a prediction
prediction = model.predict(features_scaled)

print(f"Predicted class: {prediction[0]}")


In [ ]:
import cv2
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib
from skimage import feature, measure
import pandas as pd

# Load the test image
image_path = r"C:\Users\User\Desktop\sub\final2_image.png"
image = cv2.imread(image_path)
# Define the desired input size for the CNN
input_size = (224, 224)
# Normalization
normalized_image = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

# Resize the image to match the input size of the CNN
resized_image = cv2.resize(normalized_image, input_size)

# Convert the resized image to grayscale
gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

# Calculate the color histograms for each channel
hsv_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2HSV)
hist_h = cv2.calcHist([hsv_image], [0], None, [256], [0, 256])
hist_s = cv2.calcHist([hsv_image], [1], None, [256], [0, 256])
hist_v = cv2.calcHist([hsv_image], [2], None, [256], [0, 256])

# Compute the GLCM
glcm = feature.graycomatrix(gray_image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)

# Calculate texture features
contrast = feature.graycoprops(glcm, 'contrast').ravel()[0]
dissimilarity = feature.graycoprops(glcm, 'dissimilarity').ravel()[0]
homogeneity = feature.graycoprops(glcm, 'homogeneity').ravel()[0]
energy = feature.graycoprops(glcm, 'energy').ravel()[0]

# Threshold the image to obtain a binary image
_, binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Find contours in the binary image
contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Find the largest contour by area
largest_contour = max(contours, key=cv2.contourArea)

# Calculate shape features
area = cv2.contourArea(largest_contour)
perimeter = cv2.arcLength(largest_contour, True)

# Convert the contour to a 2D image
contour_image = np.zeros_like(gray_image)
cv2.drawContours(contour_image, [largest_contour], 0, (255, 255, 255), thickness=cv2.FILLED)

# Calculate eccentricity on the 2D image
eccentricity = measure.regionprops(measure.label(contour_image))[0].eccentricity

# Calculate the convex hull of the largest contour
hull = cv2.convexHull(largest_contour)

# Calculate the area of the convex hull
hull_area = cv2.contourArea(hull)

# Calculate the perimeter of the convex hull
hull_perimeter = cv2.arcLength(hull, True)

# Calculate the solidity of the object (ratio of contour area to convex hull area)
solidity = cv2.contourArea(largest_contour) / hull_area

# Combine all features into a single feature vector
features = np.concatenate((hist_h.flatten(), hist_s.flatten(), hist_v.flatten(),
                           [contrast, dissimilarity, homogeneity, energy,
                            area, perimeter, eccentricity, hull_area,
                            hull_perimeter, solidity]))



In [ ]:
# Define paths to the scaler and model files
models = {
    "RandomForest": {
        "scaler_path": r"C:\Users\User\Desktop\exprement\random_forest_scaler.pkl",
        "model_path": r'C:\Users\User\Desktop\exprement\random_forest_model.pkl'
    },
    "LogisticRegression": {
        "scaler_path": r"C:\Users\User\Desktop\exprement\logistic_regression_scaler.pkl",
        "model_path": r'C:\Users\User\Desktop\exprement\logistic_regression_model.pkl'
    },

    "SVM": {
        "scaler_path": r"C:\Users\User\Desktop\exprement\svm_scaler.pkl",
        "model_path": r'C:\Users\User\Desktop\exprement\svm_model.pkl'
    },
    "GBM": {
        "scaler_path": r"C:\Users\User\Desktop\exprement\gbm_scaler.pkl",
        "model_path": r'C:\Users\User\Desktop\exprement\gbm_model.pkl'
    },

    "KNN": {
        "scaler_path": r"C:\Users\User\Desktop\exprement\knn_scaler.pkl",
        "model_path": r'C:\Users\User\Desktop\exprement\knn_model.pkl'
    }
}

# Iterate over the models and make predictions
for model_name, paths in models.items():
    scaler = joblib.load(paths["scaler_path"])
    features_scaled = scaler.transform(features.reshape(1, -1))
    model = joblib.load(paths["model_path"])
    prediction = model.predict(features_scaled)
    print(f"Model: {model_name}, Predicted class: {prediction[0]}")

# # Generate synthetic ground truth labels (assuming 5 classes)